In [1]:
import os
import time
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import pandas as pd
from IPython.display import display


class Info_obtain:
        
    def get_info(self, stock_ID):
        # 搜尋網站: 鉅亨
        url = 'https://www.cnyes.com/twstock/' + stock_ID
        website = requests.get(url)
        page = bs(website.text, 'html.parser')

        # 找標籤、獲取資訊
        info = page.find('div', class_='detail-content')
        info = info.select('span')
        fluctuation = page.find('div', class_='first-row')
        fluctuation = fluctuation.select('span')

        # 獲取基本資料: 股票名稱、成交價、開盤價、收盤價、最高、最低、成交量(張)
        stock = page.findAll('h2')[0].text

        open_price = info[1].text

        close_price = page.findAll('h3')[0].text

        highest = info[3].text

        lowest = info[9].text

        volume = info[5].text
        for v in volume:
            if v == ',': volume = volume.replace(v, '')

        fluc = [fluctuation[0].text, fluctuation[1].text]

        self.info_list = ['名稱: '+stock,
                          stock_ID,
                          '開盤: '+open_price,
                          '收盤: '+close_price,
                          '最高: '+highest,
                          '最低: '+lowest,
                          '交易量(張): '+volume,
                          '波動(股價): '+fluc[0],
                          '波動(百分點): '+fluc[1]
                         ]
        return self.info_list    
    
    
    def display(self):
        print('股票今日資訊:')
        for i in range(len(self.info_list)):
            if i == 1: continue
            print(self.info_list[i])
        print('='*30)
    
    
    def get_hist_data(self, stock_ID):
        url = 'https://www.cnyes.com/twstock/' + stock_ID + '/summary/full-technical-history' 
        driver = webdriver.Chrome()
        
        # 設定將想要下載的檔案儲存至現在執行的selenium資料夾裡
        params = {'behavior':'allow', 'downloadPath':os.getcwd()}
        driver.execute_cdp_cmd('Page.setDownloadBehavior', params)
        
        # 打開瀏覽器，視窗最大化
        driver.get(url)
        print('Starting WebDriver...')
        driver.maximize_window()
        time.sleep(1)
        
        try:
            # 尋找歷史資料的按鈕並下載檔案
            print('Finding history data...')
            button = driver.find_element(By.XPATH, '//*[@id="anue-ga-wrapper"]/section/div[2]/section/div[2]/div[1]/div/div/div[1]/div[1]')
            button.click()
            time.sleep(1)
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(2)
            button = driver.find_element(By.XPATH,'//*[@id="anue-ga-wrapper"]/section/div[2]/section/div[3]/div/div[2]/div/section/div[1]/div/button')
            driver.execute_script('arguments[0].scrollIntoView();',button)
            button.click()
            time.sleep(1)
            button2 = driver.find_element(By.XPATH,'//*[@id="anue-ga-wrapper"]/section/div[2]/section/div[3]/div/div[2]/div/section/div[1]/div/div[2]/div[1]/button[8]')
            button2.click()
            time.sleep(1)
            button3 = driver.find_element(By.XPATH,'//*[@id="anue-ga-wrapper"]/section/div[2]/section/div[3]/div/div[2]/div/section/div[1]/div/div[2]/div[3]/button[2]')
            button3.click()
            time.sleep(1)
            print('Downloading...')
            button_download = driver.find_element(By.XPATH,'//*[@id="anue-ga-wrapper"]/section/div[2]/section/div[3]/div/div[2]/div/section/div[1]/a/button')
            button_download.click()
            time.sleep(3)
            print('Done! Please check the file in your current working folder.')
            print('='*30)
        except:
            print('找不到此股票，請嘗試重新輸入正確的股票代號')
#         driver.quit()
            
    
    def rename_file(self, name):
        try:
            # 獲取目前正在使用的路徑和剛下載的檔案
            path = os.getcwd()
            filename = 'null_history.csv'
            file_path = os.path.join(path, filename)
            os.path.splitext(filename)

            # 如果有同名的檔案: 把原先的刪除(覆蓋掉檔案)
            if os.path.isfile(name+'_history.csv'):
                os.remove(name+'_history.csv')
            os.rename(filename, name+'_history.csv')
        except:
            print('沒有成功下載到檔案，請嘗試重新輸入正確的股票代號')
        
        
    def hist_per(self, stock_ID):
        # 網址
        url = 'https://goodinfo.tw/tw/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID='+stock_ID+'&CHT_CAT=WEEK'
        
        # 更改使用者
        website = requests.get(url, headers={'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'})
        
        # 網站讀取中文
        website.encoding = 'utf-8'
        
        # 找尋資料所在標籤
        page = bs(website.text, 'html.parser')
        trs = page.findAll('tr', {'align': 'center'})
        tds = [] # 要的資料
        for tr in trs[6:]:
            td = tr.select('td')
            tds.append(td)
        cur_price = tds[0][1].text
        for td in tds:
            del td[1:4]
        
        # 找資料的title
        title = [] # 要的title
        ths = page.findAll('th')
        for th in ths[-13:]:
            title.append(th.text)
        del title[6]
        del title[1:4]
        
        per = pd.DataFrame({
            title[0]: [td[0].text for td in tds],
            title[1]: [td[1].text for td in tds],
            title[2]: [td[2].text for td in tds],
            title[3]: [td[3].text for td in tds],
            title[4]: [td[4].text for td in tds],
            title[5]: [td[5].text for td in tds],
            title[6]: [td[6].text for td in tds],
            title[7]: [td[7].text for td in tds],
            title[8]: [td[8].text for td in tds]
        })
        return per, title, cur_price
    
    
    def PB_ratio(self, stock_ID):
        url = 'https://www.cnyes.com/twstock/'+stock_ID
        res = requests.get(url)
        page = bs(res.text, 'html.parser')
        
        # 找股價淨值比
        div = page.find('div', class_='jsx-1282029765 detail-content')
        lis = div.select('li')
        PB_ratio = lis[22].text
        PB_ratio = PB_ratio.replace('本淨比', '')
        PB_ratio = float(PB_ratio)
        if PB_ratio > 1:
            print('該支股票現在的股價淨值比為:', PB_ratio, '，股票的價格比較貴、潛在報酬較低。')
        elif PB_ratio == 1:
            print('該支股票現在的股價淨值比為:', PB_ratio, '，股票的價格中規中矩。')
        else:
            print('該支股票現在的股價淨值比為:', PB_ratio, '，股票的價格比較便宜、潛在報酬較高。')
          
        
    def financial_report(self, stock_ID):
        url = 'https://goodinfo.tw/tw/StockDetail.asp?STOCK_ID='+stock_ID
        res = requests.get(url, headers={'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'})
        res.encoding = 'utf-8'
        page = bs(res.text, 'html.parser')
        
        # 找財報評分
        table = page.find('table', class_='b1 p4_2 r10 box_shadow')
        a = table.select('a')
        a = a[1]
        a = a.text
        a = a.replace('最新','')
        a = a.replace('分','')
        a = int(a)
        span = table.select('span')
        span = span[0]
        span = span.text
        span = span.replace('平均','')
        span = span.replace('分','')
        span = int(span)
        if 0 <= a <= 40:
            print('目前財報分數為:', a, '分，差')
        elif 40 < a <= 70:
            print('目前財報分數為:', a, '分，普通')
        else:
            print('目前財報分數為:', a, '分，良好')
        if 0 <= span <= 40:
            print('歷年平均財報分數為:', span, '分，差')
        elif 40 < span <= 70:
            print('歷年平均財報分數為:', span, '分，普通')
        else:
            print('歷年平均財報分數為:', span, '分，良好')
        